In [6]:
%pip install -U --quiet opensearch-py requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import botocore

# Create a session with boto3
session = boto3.Session()

# Get the STS client
sts_client = session.client("sts")

# Get the caller identity
caller_identity = sts_client.get_caller_identity()

# Get the assumed role ARN
assumed_role_arn = caller_identity["Arn"]

# Get the role name from the assumed role ARN
role_name = assumed_role_arn.split("/")[1]

# Get the IAM client
iam_client = session.client("iam")

# Get the role details
try:
    role = iam_client.get_role(RoleName=role_name)
    iam_role_arn = role["Role"]["Arn"]
    print(f"The IAM role assumed now is: {assumed_role_arn}")
    print(f"The IAM role attached to this SageMaker notebook instance is: {iam_role_arn}")
except botocore.exceptions.ClientError as e:
    print(f"Error: {e}")

The IAM role assumed now is: arn:aws:sts::629149101570:assumed-role/opensearch-workshop-NBRole-tQMR4hqwRzQP/SageMaker
The IAM role attached to this SageMaker notebook instance is: arn:aws:iam::629149101570:role/opensearch-workshop-NBRole-tQMR4hqwRzQP


In [3]:
session = boto3.Session()
region_name = session.region_name


def get_cfn_outputs(stackname, cfn):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)["Stacks"][0]["Outputs"]:
        outputs[output["OutputKey"]] = output["OutputValue"]
    return outputs


import boto3, json

cfn = boto3.client("cloudformation", region_name)
kms = boto3.client("secretsmanager", region_name)

stackname = "opensearch-workshop"
cfn_outputs = get_cfn_outputs(stackname, cfn)

aos_credentials = json.loads(
    kms.get_secret_value(SecretId=cfn_outputs["OpenSearchSecret"])["SecretString"]
)

aos_host = cfn_outputs["OpenSearchDomainEndpoint"]
aos_host

'search-opensearch-workshop-nsyuex4f6yzuajrbfdrfxoldo4.us-west-2.es.amazonaws.com'

In [7]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import boto3
import json

credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region=region_name)

aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=600,
)

In [8]:
base_url = "https://" + aos_host + "/_plugins/_ml"
base_url

'https://search-opensearch-workshop-nsyuex4f6yzuajrbfdrfxoldo4.us-west-2.es.amazonaws.com/_plugins/_ml'

In [10]:
import requests

# Bedrock Titan Embeddings G1 - Text Connector 생성

connector_payload = {
    "name": "Amazon Bedrock Connector: embedding",
    "description": "The connector to bedrock Titan embedding model",
    "version": 1,
    "protocol": "aws_sigv4",
    "parameters": {
        "region": region_name,
        "service_name": "bedrock",
        "model": "amazon.titan-embed-text-v2:0",
    },
    "credential": {"roleArn": iam_role_arn},
    "actions": [
        {
            "action_type": "predict",
            "method": "POST",
            "url": "https://bedrock-runtime.${parameters.region}.amazonaws.com/model/${parameters.model}/invoke",
            "headers": {"content-type": "application/json", "x-amz-content-sha256": "required"},
            "request_body": '{ "inputText": "${parameters.inputText}" }',
            "pre_process_function": "connector.pre_process.bedrock.embedding",
            "post_process_function": "connector.post_process.bedrock.embedding",
        }
    ],
}

# Send the request
response = requests.post(
    base_url + "/connectors/_create",
    auth=auth,
    json=connector_payload,
)

# Print the response
print(response.text)

{"connector_id":"H_aWN5EBASM8WxlK3bAx"}


In [11]:
obj = json.loads(response.text)
connector_id = obj["connector_id"]
connector_id

'H_aWN5EBASM8WxlK3bAx'

In [12]:
register_model_group_payload = {
    "name": "Bedrock Models for titan text v2",
    "description": "This is a model group for Amazon Bedrock remote model",
}

# Send the request
response = requests.post(
    base_url + "/model_groups/_register",
    auth=auth,
    json=register_model_group_payload,
)

# Print the response
print(response.text)

{"model_group_id":"IPaXN5EBASM8WxlKQLCx","status":"CREATED"}


In [13]:
obj = json.loads(response.text)
model_group_id = obj["model_group_id"]
model_group_id

'IPaXN5EBASM8WxlKQLCx'

In [14]:
register_model_payload = {
    "name": "Bedrock text embedding model",
    "function_name": "remote",
    "model_group_id": model_group_id,
    "description": "This is Bedrock Titan Embeddings V2 - Text model",
    "connector_id": connector_id,
}

# Send the request
response = requests.post(
    base_url + "/models/_register",
    auth=auth,
    json=register_model_payload,
)

# Print the response
print(response.text)

{"task_id":"IfagN5EBASM8WxlKNbBj","status":"CREATED","model_id":"IvagN5EBASM8WxlKNbB8"}


In [15]:
obj = json.loads(response.text)
model_id = obj["model_id"]
print(model_id)

IvagN5EBASM8WxlKNbB8


In [16]:
deploy_model_payload = {}

# Send the request
response = requests.post(
    base_url + "/models/" + model_id + "/_deploy",
    auth=auth,
    json=deploy_model_payload,
)

# Print the response
print(response.text)

{"task_id":"I_agN5EBASM8WxlKdrAm","task_type":"DEPLOY_MODEL","status":"COMPLETED"}


In [17]:
predict_model_payload = {"parameters": {"inputText": "hello goodbye"}}

# Send the request
response = requests.post(
    base_url + "/models/" + model_id + "/_predict",
    auth=auth,
    json=predict_model_payload,
)

# Print the response
len(response.json()["inference_results"][0]["output"][0]["data"])

1024

In [ ]:
%store model_id